In [1]:
from modeling.arghandling import flatten_args, unflatten_args
from modeling.arghandling import encode, decode, Encoding, EncodedFunction
import numpy as np

In [2]:
x = [1, np.array([1,2]), np.array([[4.,5],[6,7]])]

In [3]:
{'a': np.array([1,2,3.])}

{'a': array([1., 2., 3.])}

In [4]:
np.array([1,2,3.]).dtype

dtype('float64')

In [5]:
v = flatten_args(x)
v

DeviceArray([1., 1., 2., 4., 5., 6., 7.], dtype=float32)

In [6]:
unflatten_args(v, shapes=((1,), (2,), (2,2)), convertscalars=True, tonumpy=True)

(1.0,
 array([1., 2.]),
 array([[4., 5.],
        [6., 7.]]))

In [7]:
w = encode({'a': 1, 'b': np.array([[4,5],[6,7]]), 'c': 3}, 
        ('c', 'b', 'a'), flatten=True)
w

DeviceArray([3, 4, 5, 6, 7, 1], dtype=int32)

In [10]:
decode(w, order=['c', 'b', 'a'], shapes=((1,), (2,2), (1,)), 
       unflatten=True, cleanup=True, tonumpy=True)

{'c': 3.0,
 'b': array([[4., 5.],
        [6., 7.]]),
 'a': 1.0}

In [9]:
f = lambda x, y, z: (x**2+z+y+np.exp(-y), np.array([x,y,z]))

## Encoded Functions

In [10]:
Ein = Encoding(('x','y','z'))
Eout = Encoding(('a','b'), shapes=((1,), (3,)))

In [11]:
Ein.encode({'x': 1, 'y': 2, 'z': 3})

(1, 2, 3)

In [12]:
dict_in_dict_out = lambda d: Eout.decode(f(*Ein.encode(d)))
dict_in_flat_out = lambda d: flatten_args(f(*Ein.encode(d)))

In [13]:
f(*Ein.encode({'x': 1, 'y': 2, 'z': 3}))

(6.135335283236612, array([1, 2, 3]))

In [14]:
dict_in_dict_out({'x': 1, 'y': 2, 'z': 3})

{'a': 6.135335283236612, 'b': array([1, 2, 3])}

In [15]:
residuals = dict_in_flat_out({'x': 1, 'y': 2, 'z': 3})
residuals

DeviceArray([6.1353354, 1.       , 2.       , 3.       ], dtype=float32)

In [16]:
Eout.decode(residuals, unflatten=True, cleanup=True, tonumpy=True)

{'a': 6.135335445404053, 'b': array([1., 2., 3.])}

## Operate with encoders

In [17]:
def reverse_encoding(encoder):
    mapping = dict()
    parent = encoder.parent
    while parent is not None:
        mapping = dict(zip(encoder.order, parent.order))
        parent = parent.parent
    return mapping

In [19]:
E1 = Encoding(('A','B','C'))
E2 = Encoding(('x','y','z'), parent=E1)
E3 = Encoding((1,2,3), parent=E2)

In [21]:
reverse_encoding(E3)

{1: 'A', 2: 'B', 3: 'C'}

In [58]:
def compose(f, g=None, mapping=None, parent_encoder=False):
    mapping = mapping if mapping is not None else dict()
    new_decoder = g.decoder if g is not None else f.decoder
    if parent_encoder:
        mapping = reverse_encoding(f.encoder)
        mapping.update(reverse_encoding(new_decoder))
        mapping = mapping.get
    func = f.f
    if g is not None:
        inter_order_out = tuple(mapping(key,key) for key in f.decoder.order)
        inter_order_in= tuple(mapping(key,key) for key in g.encoder.order)
        F = EncodedFunction(f, f.encoder, inter_order_in)
        G = EncodedFunction(g, inter_order_out, g.decoder)
        func = lambda *args: G.dict_in_only(F.dict_out_only(*args))   
    new_encoder_order = tuple(mapping(key,key) for key in f.encoder.order)
    new_encoder = Encoding(new_encoder_order, parent=f.encoder)
    new_decoder_order = tuple(mapping(key,key) for key in new_decoder.order)
    new_decoder = Encoding(new_decoder_order, parent=new_decoder)
    return EncodedFunction(func, new_encoder, new_decoder)

In [59]:
F = EncodedFunction(f, Ein, Eout)

In [60]:
F.dict_in_dict_out({'x': 1, 'y': 2, 'z': 3})

{'a': 6.135335283236612, 'b': array([1, 2, 3])}

In [61]:
#G = compose(F, mapping={'x':1, 'y':2, 'z':3, 'a':4}.get)
G = compose(F, mapping=lambda x,_: 'x_{}'.format(['x','y','z','a','b'].index(x)))

In [62]:
G.dict_in_dict_out({'x_0': 1, 'x_1': 2, 'x_2': 3})

{'x_3': 6.135335283236612, 'x_4': array([1, 2, 3])}

In [63]:
H = compose(G, parent_encoder=True)

In [64]:
H.dict_in_dict_out({'x': 1, 'y': 2, 'z': 3})

{'a': 6.135335283236612, 'b': array([1, 2, 3])}